# 텍스트 분류 : 구분 → 원인 분류

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
from konlpy.tag import Okt, Kkma, Mecab, Hannanum
Okt = Okt()
kkma = Kkma()
hannanum = Hannanum()

In [3]:
df = pd.read_csv('final_text_data2.csv', sep='\t', encoding='UTF-8')

In [4]:
df.columns

Index(['std_clm_type_cd', 'ro_y', 'clm_jgmt_dt', 'ceq_caus_cd', 'ceq_psit_cd',
       'ceq_psit_nm', 'trim_plnt_cd', 'prj_vehl_cd', 'mdl_cd', 'part_sys_cd',
       'repn_part_cd', 'clm_prtl_sbc', '대상 선정', '구분', '책임구', '원인', '검토결과'],
      dtype='object')

In [5]:
df.head(3)

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd    ceq_psit_nm  \
0               W  2017     20170811         ZZ7         B11  조립 문제_미장착/오장착   
1               W  2018     20180822         ZZ7         B11  조립 문제_미장착/오장착   
2               W  2018     20181006         ZZ5         B11  조립 문제_미장착/오장착   

  trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
0         KV11          CK     J5          TT        82305   
1         KV11          YP     A9          TT        84211   
2         KV11          YP     A9          TT        83912   

                                        clm_prtl_sbc 대상 선정   구분  책임구   원인  \
0  [C] 1.현상:엔진룸 다량의 연료 누유 발생.2.점검:엔진룸 다량의 연료 누유 점...     X  NaN  NaN  NaN   
1                       [C] 1.현상: 리어 도어 앤티 취핑 필림 미장착     X  NaN  NaN  NaN   
2  [C] 1.현상:Rr RH 커튼 후크  미장착 출고됨.2.조치내용:Rr RH 커튼 ...     X  NaN  NaN  NaN   

   검토결과  
0  X---  
1  X---  
2  X---

In [6]:
df.shape

(10629, 17)

# null인 값 확인 및 제거

In [7]:
df_bak = df.copy()

In [8]:
# 제거
# df = df[df['대상 선정'].notnull()].copy()
# 다른값으로 채우기
df = df.fillna('N')

In [9]:
df.shape

(10629, 17)

In [10]:
category_count = df['대상 선정'].value_counts()
category_count

X    5519
O    3013
N    2097
Name: 대상 선정, dtype: int64

In [11]:
category_count = df['구분'].value_counts()
category_count

N          7617
기능(커넥터)    1601
실내부품이음     1050
하체이음        361
Name: 구분, dtype: int64

In [12]:
category_count = df['책임구'].value_counts()
category_count

N       7617
구매      2274
생산공장     724
연구소       14
Name: 책임구, dtype: int64

In [13]:
category_count = df['원인'].value_counts()
category_count

N        7617
부품       2274
조립불량      230
결선불량      225
체결불량      103
기타        100
미결선        37
이종/간섭      21
설계/구조      14
이물          8
Name: 원인, dtype: int64

## 1-1대상선정) 'O'값 추출

In [14]:
dfo = df.loc[df['대상 선정']=='O',:] # 대상 선정값이 O 인 것만 가져오기 (X제외)
dfo.shape

(3013, 17)

## 1-2대상선정) 'X'값 추출

In [15]:
dfx = df.loc[df['대상 선정']=='X',:] # 대상 선정값이 X인 것만 가져오기 (O제외)
dfx.shape

(5519, 17)

##    2-1구분) '하체이음'값 추출 + 'X'값 더하기

In [16]:
df_hache1 = dfo.loc[dfo['구분']=='하체이음',:]
df_hache1.shape

(361, 17)

In [17]:
df_hache = pd.concat([df_hache1,dfx])
df_hache.shape

(5880, 17)

In [18]:
category_count1 = df_hache['원인'].value_counts()
category_count1

N       5519
부품       249
기타        53
체결불량      33
조립불량      26
Name: 원인, dtype: int64

## 2-2구분) '실내부품이음'값 추출 + 'X'값 더하기

In [19]:
df_silne1 = dfo.loc[dfo['구분']=='실내부품이음',:]
df_silne1.shape

(1050, 17)

In [20]:
df_silne = pd.concat([df_silne1,dfx])
df_silne.shape

(6569, 17)

In [140]:
category_count2 = df_silne['원인'].value_counts()
category_count2

N        5519
부품        737
조립불량      204
체결불량       70
기타         17
설계/구조      14
이물          8
Name: 원인, dtype: int64

## 2-3구분) '기능(커넥터)'값 추출 + 'X'값 더하기

In [21]:
df_connect1 = dfo.loc[dfo['구분']=='기능(커넥터)',:]
df_connect1.shape

(1601, 17)

In [22]:
df_connect = pd.concat([df_connect1,dfx])
df_connect.shape

(7120, 17)

In [141]:
category_count3 = df_connect['원인'].value_counts()
category_count3

N        5519
부품       1288
결선불량      225
미결선        37
기타         30
이종/간섭      21
Name: 원인, dtype: int64

# '하체이음'값 '원인'분류 모델링

In [23]:
df_hache = df_hache.reindex()
df_silne = df_silne.reindex()
df_connect = df_connect.reindex()

# 전처리

In [24]:
# 예측값과 실제값 비교를 위해 컬럼을 하나 더 생성합니다.
df_hache['cause_pred'] = df_hache['원인'].astype(str).copy()
df_silne['cause_pred'] = df_silne['원인'].astype(str).copy()
df_connect['cause_pred'] = df_connect['원인'].astype(str).copy()

In [25]:
def preprocessing(text):
    # 개행문자 제거
    text= str(text)
    text = re.sub('\\\\n', ' ', text) # 특수문자 제거
    #text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)   # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    #text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)   # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)   # 한글, 영문만 남기고 모두 제거하도록 합니다.
    return text

In [26]:
# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['현상', '[C]','현상및점검','현    상','현 상','점검점검','점검내용','조치내용','점검점검내용','현상및조치','C','원인',
             '점검/원인','사진첨부','(사진첨부)','사진참조','점검및원인','동영상참조','동영상','대리점','고객','고객에게','상기',
             '차량','상기차량은','자동차','당차량은','당차량','당','차량은','점검및조치내용','F/LH','F/RH','L/H','R/H','L/H리어',
             'R/H리어','LH/RH','LH','RH','(RH)','(LH)','R/LH','R/RH','리어RH','리어LH','LH쪽','RH쪽','Rr','RR','FRT','Frt',
             'frt','rr','C-10000','C-100','(좌)','(우)','A/S']
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words) #stops에 안 걸리는 나머지 것들 이어붙이기

In [27]:
%time df_hache['clm_prtl_sbc'] = df_hache['clm_prtl_sbc'].apply(preprocessing)

Wall time: 68 ms


In [28]:
%time df_silne['clm_prtl_sbc'] = df_silne['clm_prtl_sbc'].apply(preprocessing)

Wall time: 77 ms


In [29]:
%time df_connect['clm_prtl_sbc'] = df_connect['clm_prtl_sbc'].apply(preprocessing)

Wall time: 79 ms


In [30]:
%time df_hache['clm_prtl_sbc'] = df_hache['clm_prtl_sbc'].apply(remove_stopwords)

Wall time: 169 ms


In [31]:
%time df_silne['clm_prtl_sbc'] = df_silne['clm_prtl_sbc'].apply(remove_stopwords)

Wall time: 182 ms


In [32]:
%time df_connect['clm_prtl_sbc'] = df_connect['clm_prtl_sbc'].apply(remove_stopwords)

Wall time: 199 ms


In [33]:
df_hache['clm_prtl_sbc'].head(10)

209            운행중 가속시 부르릉 소음으로 입고함  점검 및 조치    배기파이프 ...
210       주행 가속시 떨림음발생및 진동 발생건인터쿨러 마운팅 브라켓 고정볼트 미조립건 발...
1932       로암 캠볼트 조임불량으로 인하여 주행시 딸그락딸그락 하는 소음및 타이어 편마모...
1948        이음 발생으로 입고       RPM 연료호스 크램프 이음부 차체 간섭발생으...
1985      고속주행 좌커브 주행시나 움벅 파인도로 주행시 탁탁    소음발생점검 바디리어 ...
2114          제동시 소음발생으로 입고   및  시운전시 천천히 제동시 끼익 소음발생으...
2144        후진시 운전석 뒤 타이어 드럼쪽에서 소리남으로 입고  점검 후진 점검 확인리...
2516       주행중 정차 후진시 타이어 뒤로 회전이 안되면서 탁소리가 나고 타는냄새도 나면...
3809        후진시 소음발생  점검    후진시 브레이크 걸렸다가 풀릴 때  딱   이음...
3811        차량후진시 리어좌측 샤시부분에서 뚜 둑하는 소음지속발생으로 입고   제동하고...
Name: clm_prtl_sbc, dtype: object

In [34]:
df_silne['clm_prtl_sbc'].head(10)

15       테일게이트트림에서 뜨르르르 이상소음발생으로 테일게이트트림 탈착하여 확인시 클립이...
40        상기차량 주행중 운전석 시트가 들석이는 현상발생입고 점검시 운전석 앞 우측 시...
66       체인지 레버 N레인지 에서 D 레인지 변경시 찍찍 이음 발생레바부위 소음 확인위...
164                조립불량으로 너트가 끼어 창문 내리거나 올릴때  드르륵  소리 발생
171      브레이크 마스터 실린더 로드 와브레이크 페달 연결 핀 조립 않되어 있음  부품 ...
179      주행시 소음으로 입고시운전및 점검결과 테일게이트 래치관련 이음테일게이트 트립 탈...
220          조수석 시트 들썩거림  점검   조수석 리어 마운틴 볼트 미체결 되어있음...
276         리어슬라이딩  도어레일 커버소음및유격발생으로 입고   점검  레일커버 클립...
304      프론트 필라 좌측 트림 교환후 간섭이음으로 재교환 작업원인 프론트 필라 성형불량...
323        주행중 요철부위 통과시 및 선회시 리어측 이음발생점검   시운전 점검중 요철...
Name: clm_prtl_sbc, dtype: object

In [35]:
df_connect['clm_prtl_sbc'].head(10)

12          가  인도후 오디오 작동시 주파수 및 노이즈 발생으로 불만 제기   및 ...
14          주행중 엔진경고등 점등   및  스캔점검결과 P       DPF효율저하...
19                     조수석사이드에어백 경고등 점등됨 컨넥터 체결불량으로 재체결
24                        USB 작동 안됨원인 USB 컨넥터 조립 않됨 재조립
25                       어라운드뷰 장착차량인데 운전석 미러에는 카메라 미장착됨
32        리어LH윈도우작동불량  점검시 뒤운전석쪽만 작동불량 퓨즈확인시정상   트림탈...
38      운전석 뒤도어 유리창 작동 안되어  메인파워스위치 배선인가 점검후뒷도어트림스위치...
58           와이퍼 스몰등 후방카메라 오작동 불량   및   리어 루프 배선 R  ...
63               출고후 DMB작동불량으로 입고됨   내용        위 현상으...
77       운전석 뒤쪽 창문이 열리지 않는다며 점검의뢰  창문 작동 점검시 모터에 전원공...
Name: clm_prtl_sbc, dtype: object

# 학습데이터와 테스트데이터 만들기

In [36]:
df_hache = df_hache.reindex() #index 재설정. reindex:새로운 색인에 맞도록 객체를 새로 생성하는 기능.
df_silne = df_silne.reindex()
df_connect = df_connect.reindex()

In [37]:
df_hache.shape

(5880, 18)

In [38]:
df_silne.shape

(6569, 18)

In [39]:
df_connect.shape

(7120, 18)

In [40]:
split_count1 = int(df_hache.shape[0] * 0.9)
split_count1

5292

In [41]:
df_train1 = df_hache[:split_count1].copy()
df_train1.shape

(5292, 18)

In [42]:
df_test1 = df_hache[split_count1:].copy()
df_test1.shape

(588, 18)

In [43]:
split_count2 = int(df_silne.shape[0] * 0.9)
split_count2

5912

In [44]:
df_train2 = df_silne[:split_count2].copy()
df_train2.shape

(5912, 18)

In [45]:
df_test2 = df_silne[split_count2:].copy()
df_test2.shape

(657, 18)

In [46]:
split_count3 = int(df_connect.shape[0] * 0.9)
split_count3

6408

In [47]:
df_train3 = df_connect[:split_count3].copy()
df_train3.shape

(6408, 18)

In [48]:
df_test3 = df_connect[split_count3:].copy()
df_test3.shape

(712, 18)

# 단어 벡터화

In [85]:
from sklearn.feature_extraction.text import CountVectorizer

stops = ['[C]']
vectorizer = CountVectorizer(analyzer = 'word', # 캐릭터 단위로 벡터화 할 수도 있습니다.
                             tokenizer = None, # 토크나이저를 따로 지정해 줄 수도 있습니다.
                             preprocessor = None, # 전처리 도구
                             stop_words = stops, # 불용어 nltk등의 도구를 사용할 수도 있습니다.
                             min_df = 1, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋다. 
                             ngram_range=(1, 3), # BOW의 단위를 1~3개로 지정합니다.
                             max_features = 20000 # 만들 피처의 수, 단어의 수가 된다.
                            )
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words=['[C]'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [86]:
#%%time 
train_feature_vector1 = vectorizer.fit_transform(df_train1['clm_prtl_sbc'])
train_feature_vector1.shape

(5292, 20000)

In [87]:
%%time 
test_feature_vector1= vectorizer.fit_transform(df_test1['clm_prtl_sbc'])
test_feature_vector1.shape

Wall time: 83 ms


In [88]:
#%%time 
train_feature_vector2 = vectorizer.fit_transform(df_train2['clm_prtl_sbc'])
train_feature_vector2.shape

(5912, 20000)

In [89]:
%%time 
test_feature_vector2= vectorizer.fit_transform(df_test2['clm_prtl_sbc'])
test_feature_vector2.shape

Wall time: 100 ms


In [90]:
#%%time 
train_feature_vector3 = vectorizer.fit_transform(df_train3['clm_prtl_sbc'])
train_feature_vector3.shape

(6408, 20000)

In [91]:
%%time 
test_feature_vector3= vectorizer.fit_transform(df_test3['clm_prtl_sbc'])
test_feature_vector3.shape

Wall time: 109 ms


In [92]:
vocab1 = vectorizer.get_feature_names()
print(len(vocab1))
vocab1[:10]

20000


['abs',
 'abs 경고등',
 'abs 경고등 점등',
 'ass',
 'ass 교환',
 'avm',
 'avm 수동',
 'avm 수동 보정',
 'avm조정함',
 'a필라']

In [93]:
vocab2 = vectorizer.get_feature_names()
print(len(vocab2))
vocab2[:10]

20000


['abs',
 'abs 경고등',
 'abs 경고등 점등',
 'ass',
 'ass 교환',
 'avm',
 'avm 수동',
 'avm 수동 보정',
 'avm조정함',
 'a필라']

In [94]:
vocab3 = vectorizer.get_feature_names()
print(len(vocab3))
vocab3[:10]

20000


['abs',
 'abs 경고등',
 'abs 경고등 점등',
 'ass',
 'ass 교환',
 'avm',
 'avm 수동',
 'avm 수동 보정',
 'avm조정함',
 'a필라']

In [95]:
dist1 = np.sum(train_feature_vector1, axis=0)
word_cnt1 = pd.DataFrame(dist1, columns=vocab1)
word_cnt1

abs  abs 경고등  abs 경고등 점등  ass  ass 교환  avm  avm 수동  avm 수동 보정  avm조정함  a필라  \
0   28        2           2    4       2    4       2          2       3    2   

   ...    흡입호스 재조립  흡입호스 재조립 개선  흡집  흡집 보임  흡집 보임 점검  히터  히터 작동시  히터 작동시 소음  \
0  ...           7            2   2      2         3   2       5          2   

   힌지  힌지 우측  
0  23      2  

[1 rows x 20000 columns]

In [96]:
dist2 = np.sum(train_feature_vector2, axis=0)
word_cnt2 = pd.DataFrame(dist2, columns=vocab2)
word_cnt2

abs  abs 경고등  abs 경고등 점등  ass  ass 교환  avm  avm 수동  avm 수동 보정  avm조정함  a필라  \
0    1       27           2    2       3    4       2          2       3    2   

   ...    흡입호스 재조립  흡입호스 재조립 개선  흡집  흡집 보임  흡집 보임 점검  히터  히터 작동시  히터 작동시 소음  \
0  ...           2            2   3      2         5   2      26          2   

   힌지  힌지 우측  
0   2      2  

[1 rows x 20000 columns]

In [97]:
dist3 = np.sum(train_feature_vector3, axis=0)
word_cnt3 = pd.DataFrame(dist3, columns=vocab3)
word_cnt3

abs  abs 경고등  abs 경고등 점등  ass  ass 교환  avm  avm 수동  avm 수동 보정  avm조정함  a필라  \
0   55        4           5    3      10    3       4          6       3    4   

   ...    흡입호스 재조립  흡입호스 재조립 개선  흡집  흡집 보임  흡집 보임 점검  히터  히터 작동시  히터 작동시 소음  \
0  ...           2            2   3      2         2   4       9          3   

   힌지  힌지 우측  
0   5     22  

[1 rows x 20000 columns]

# tf-idf 가중치 적용

In [51]:
# tf-idf(Term Frequency-Inverse Document Frequency). 가중치를 구하는 알고리즘.

In [98]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
transformer

TfidfTransformer(norm='l2', smooth_idf=False, sublinear_tf=False,
         use_idf=True)

In [99]:
#%%time 
train_feature_tfidf1 = transformer.fit_transform(train_feature_vector1)
train_feature_tfidf1.shape

(5292, 20000)

In [100]:
%%time 
test_feature_tfidf1 = transformer.fit_transform(test_feature_vector1)
test_feature_tfidf1.shape

Wall time: 40 ms


In [101]:
#%%time 
train_feature_tfidf2 = transformer.fit_transform(train_feature_vector2)
train_feature_tfidf2.shape

(5912, 20000)

In [102]:
%%time 
test_feature_tfidf2 = transformer.fit_transform(test_feature_vector2)
test_feature_tfidf2.shape

Wall time: 66 ms


In [103]:
#%%time 
train_feature_tfidf3 = transformer.fit_transform(train_feature_vector3)
train_feature_tfidf3.shape

(6408, 20000)

In [104]:
%%time 
test_feature_tfidf3 = transformer.fit_transform(test_feature_vector3)
test_feature_tfidf3.shape

Wall time: 47 ms


# 머신러닝 : 학습

In [105]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 분류기를 사용
forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state=2018)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=2018, verbose=0,
            warm_start=False)

In [106]:
# 학습에 사용할 y_label 을 넣어줍니다.
# 어떤 분야의 청원인지 예측할 것이기 때문에 category를 넣어줍니다.
# y_label2 = df_train['caus_pred']
# y_label3 = df_train['charge_pred']
y_label1 = df_train1['cause_pred']
%time forest = forest.fit(train_feature_tfidf1, y_label1)
# %time forest2 = forest.fit(train_feature_tfidf, y_label2)
# %time forest3 = forest.fit(train_feature_tfidf, y_label3)

Wall time: 706 ms


In [107]:
# 학습에 사용할 y_label 을 넣어줍니다.
# 어떤 분야의 청원인지 예측할 것이기 때문에 category를 넣어줍니다.
# y_label2 = df_train['caus_pred']
# y_label3 = df_train['charge_pred']
y_label2 = df_train2['cause_pred']
%time forest = forest.fit(train_feature_tfidf2, y_label2)
# %time forest2 = forest.fit(train_feature_tfidf, y_label2)
# %time forest3 = forest.fit(train_feature_tfidf, y_label3)

Wall time: 1.25 s


In [108]:
# 학습에 사용할 y_label 을 넣어줍니다.
# 어떤 분야의 청원인지 예측할 것이기 때문에 category를 넣어줍니다.
# y_label2 = df_train['caus_pred']
# y_label3 = df_train['charge_pred']
y_label3 = df_train3['cause_pred']
%time forest = forest.fit(train_feature_tfidf3, y_label3)
# %time forest2 = forest.fit(train_feature_tfidf, y_label2)
# %time forest3 = forest.fit(train_feature_tfidf, y_label3)

Wall time: 1.4 s


# 모델평가

In [109]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

scoring = 'accuracy'
%time score1 = cross_val_score(forest, train_feature_vector1, y_label1, cv=k_fold, n_jobs=-1, scoring=scoring)
score1

Wall time: 26.9 s


array([0.93862134, 0.94523135, 0.93572779, 0.94328922, 0.94234405])

In [110]:
round(np.mean(score1)*100,2)

94.1

In [111]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

scoring = 'accuracy'
%time score2 = cross_val_score(forest, train_feature_vector2, y_label2, cv=k_fold, n_jobs=-1, scoring=scoring)
score2

Wall time: 13.9 s


array([0.84699915, 0.84277261, 0.8037225 , 0.83502538, 0.84094755])

In [112]:
round(np.mean(score2)*100,2)

83.39

In [113]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

scoring = 'accuracy'
%time score3 = cross_val_score(forest, train_feature_vector3, y_label3, cv=k_fold, n_jobs=-1, scoring=scoring)
score3

Wall time: 12.8 s


array([0.82449298, 0.83385335, 0.81123245, 0.82903981, 0.82982045])

In [114]:
round(np.mean(score3)*100,2)

82.57

# 예측

In [115]:
# 테스트 데이터를 넣고 예측합니다.
y_pred1 = forest.predict(test_feature_vector1)
y_pred1[:3]

array(['N', 'N', 'N'], dtype=object)

In [116]:
y_pred1.shape

(588,)

In [117]:
# 예측 결과를 저장하기 위해 데이터프레임에 담아 줍니다.
output1 = pd.DataFrame(data={'cause_pred':y_pred1})
output1.head(10)

cause_pred
0          N
1          N
2          N
3          N
4          N
5          N
6         부품
7          N
8          N
9          N

In [118]:
df_test1['cause_pred'] = y_pred1

In [119]:
# 테스트 데이터를 넣고 예측합니다.
y_pred2 = forest.predict(test_feature_vector2)
y_pred2[:3]

array(['부품', '부품', '부품'], dtype=object)

In [120]:
y_pred2.shape

(657,)

In [121]:
# 예측 결과를 저장하기 위해 데이터프레임에 담아 줍니다.
output2 = pd.DataFrame(data={'cause_pred':y_pred2})
output2.head(10)

cause_pred
0         부품
1         부품
2         부품
3         부품
4          N
5          N
6          N
7          N
8          N
9          N

In [122]:
df_test2['cause_pred'] = y_pred2

In [123]:
# 테스트 데이터를 넣고 예측합니다.
y_pred3 = forest.predict(test_feature_vector3)
y_pred3[:3]

array(['N', 'N', 'N'], dtype=object)

In [124]:
y_pred3.shape

(712,)

In [125]:
# 예측 결과를 저장하기 위해 데이터프레임에 담아 줍니다.
output3 = pd.DataFrame(data={'cause_pred':y_pred3})
output3.head(10)

cause_pred
0          N
1          N
2          N
3          N
4          N
5          N
6          N
7          N
8          N
9          N

In [126]:
df_test3['cause_pred'] = y_pred3

# 결과검증

In [127]:
df_test1['pred_diff'] = 0
df_test1['pred_diff'] = (df_test1['원인'] == df_test1['cause_pred'] ) == 1
df_test1['pred_diff'] = df_test1['pred_diff'].astype(int)
df_test1.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd ceq_psit_nm  \
9535               W  2018     20181025         ZZ3         Q57   소음/이음_작동시   
9537               W  2016     20160907         ZZ3         Q57   소음/이음_작동시   
9538               W  2017     20170916         ZZ3         Q57   소음/이음_작동시   
9539               W  2015     20151126         ZZ3         Q57   소음/이음_작동시   
9541               W  2015     20150325         ZZ3         Q57   소음/이음_작동시   

     trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
9535         KV11          CK     J5          TT        98350   
9537         KV11          YP     A9          TT        83940   
9538         KV11          YP     A9          TT        83940   
9539         KV11          YP     A9          TT        83810   
9541         KV11          YP     A9          BD        76003   

                                           clm_prtl_sbc 대상 선정 구분 책임구 원인  검토결과  \
9535      와이퍼브러쉬 작동시 소음 발생  유리 유막 제거후 에도 소음 계속 발생됨  ...     X  N   N  N  X---   
9537    JX          W       슬라이딩 도어 작동시 소음으로 입고원인 슬라...     X  N   N  N  X---   
9538       우측 슬라이딩도어 작동시 간헐적 소음 찌익    센터롤러 소음으로 판단되어 교환     X  N   N  N  X---   
9539    파워슬라이딩 도어 어셈블리 좌측 소손에 의한 작동중 삑삑삑  이음 발생됨점검후 ...     X  N   N  N  X---   
9541    프론트 도어 모듈패널 교환  운전석 윈도우 하강시     지점부터 항시 소음발생...     X  N   N  N  X---   

     cause_pred  pred_diff  
9535          N          1  
9537          N          1  
9538          N          1  
9539          N          1  
9541          N          1

In [128]:
diff_sum1 = df_test1['pred_diff'].sum()
diff_sum1

467

In [129]:
y_pred_len1 = len(y_pred1)
y_pred_len1

588

In [130]:
df_test2['pred_diff'] = 0
df_test2['pred_diff'] = (df_test2['원인'] == df_test2['cause_pred'] ) == 1
df_test2['pred_diff'] = df_test2['pred_diff'].astype(int)
df_test2.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd ceq_psit_nm  \
9441               W  2018     20180920         ZZ3         Q57   소음/이음_작동시   
9443               W  2015     20150119         ZZ3         Q57   소음/이음_작동시   
9444               W  2017     20171213         ZZ5         Q57   소음/이음_작동시   
9445               W  2016     20160202         ZZ3         Q57   소음/이음_작동시   
9446               W  2017     20170919         ZZ3         Q57   소음/이음_작동시   

     trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
9441         KV11          YP     A9          TT        98350   
9443         KV11          YP     A9          TT        82210   
9444         KV11          YP     A9          TT        98350   
9445         KV11          YP     A9          AC        93480   
9446         KV11          YP     A9          TT        81770   

                                           clm_prtl_sbc 대상 선정 구분 책임구 원인  검토결과  \
9441    와이퍼 작동시 하강시에 소음발생및 중앙부 안닦힘 드드득   조치 운전석 와이퍼 ...     X  N   N  N  X---   
9443      윈도우 글라스 상 하 작동 시 소음점검   윈도우 글라스 소음 유무 확인점검...     X  N   N  N  X---   
9444        와이퍼 잘안닦임으로 입고  점검내용및원인  전면유리 이물질 및데미지 않음...     X  N   N  N  X---   
9445     ABS 경고등 점등 점검 조향각 센서 작동 불량으로 인한 경고등 점등 조치 조...     X  N   N  N  X---   
9446                                태일게이트열림시소음발생되어교환조치함     X  N   N  N  X---   

     cause_pred  pred_diff  
9441         부품          0  
9443         부품          0  
9444         부품          0  
9445         부품          0  
9446          N          1

In [131]:
diff_sum2 = df_test2['pred_diff'].sum()
diff_sum2

558

In [132]:
y_pred_len2 = len(y_pred2)
y_pred_len2

657

In [133]:
df_test3['pred_diff'] = (df_test3['원인'] == df_test3['cause_pred'] ) == 1
df_test3['pred_diff'] = df_test3['pred_diff'].astype(int)
df_test3.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd ceq_psit_nm  \
9376               W  2015     20150408         ZZ5         Q57   소음/이음_작동시   
9378               W  2015     20150217         ZZ5         Q57   소음/이음_작동시   
9379               W  2016     20161019         ZZ5         Q57   소음/이음_작동시   
9381               W  2017     20170628         ZZ5         Q57   소음/이음_작동시   
9382               W  2018     20180706         ZZ5         Q57   소음/이음_작동시   

     trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
9376         KV11          YP     A9          TT        81770   
9378         KV11          YP     A9          TT        82530   
9379         KV11          YP     A9          TT        83940   
9381         KV11          YP     A9          TT        84660   
9382         KV11          YP     A9          TT        98350   

                                           clm_prtl_sbc 대상 선정 구분 책임구 원인  검토결과  \
9376        파워 테일게이트 닫을시 부드럽지 않으며 세게 닫히며 튕김  점검   수동...     X  N   N  N  X---   
9378             글라스런체널작동시소음건프론트좌측 글라스런체널 상하작동시소음발생으로교환     X  N   N  N  X---   
9379                                싸이드 도어 작동후 닫힘시 이음발생     X  N   N  N  X---   
9381                      콘솔암레스트에 팔을 올려놓고 주행중 소음  삐걱삐걱      X  N   N  N  X---   
9382           와이퍼 작동시 소음  와이퍼 블레이드 부품불량  와이퍼 블레이드 부품교환     X  N   N  N  X---   

     cause_pred  pred_diff  
9376          N          1  
9378          N          1  
9379          N          1  
9381          N          1  
9382          N          1

In [134]:
diff_sum3 = df_test3['pred_diff'].sum()
diff_sum3

611

In [135]:
y_pred_len3 = len(y_pred3)
y_pred_len3

712

# 정확도

In [136]:
Acy1 = round(diff_sum1/y_pred_len1*100,1)
Acy1

79.4

In [137]:
Acy2 = round(diff_sum2/y_pred_len2*100,1)
Acy2

84.9

In [138]:
Acy3 = round(diff_sum3/y_pred_len3*100,1)
Acy3

85.8